In [396]:
import pandas as pd
import geopandas as gpd
from pyproj import CRS
from keplergl import KeplerGl
from shapely.geometry import Point, LineString

In [397]:
fps = r'data/OD_matrix_201607.csv'
geodata_summer = gpd.read_file(fps)

In [398]:
# define crs
geodata_summer.crs = CRS.from_epsg(3301)

In [399]:
geodata_summer.crs.name

'Estonian Coordinate System of 1997'

In [400]:
# adding geometry
xorigins = geodata_summer['X_start'].to_list()
yorigins = geodata_summer['Y_start'].to_list()

geodata_summer['geometry'] = [Point(float(xcoor), float(ycoor)) for xcoor, ycoor in zip(xorigins, yorigins)]

geodata_summer.head()

,KANT_start,KANT_end,start_kant_id,end_kant_id,X_start,Y_start,X_end,Y_end,route_id,Population,RegularMovers,geometry
0,Aakre Valga,Aakre Valga,1,1,627213.008798677,6440209.31365088,627213.008798677,6440209.31365088,1697,470,97,POINT (627213.009 6440209.314)
1,Aakre Valga,Elva linn,1,49,627213.008798677,6440209.31365088,642136.014360787,6456562.83692973,41506,470,8,POINT (627213.009 6440209.314)
2,Aakre Valga,Haabersti linnaosa,1,840,627213.008798677,6440209.31365088,535384.478080503,6587643.15187738,710636,470,1,POINT (627213.009 6440209.314)
3,Aakre Valga,Helme Valga,1,82,627213.008798677,6440209.31365088,611392.704776615,6429856.66892107,69457,470,1,POINT (627213.009 6440209.314)
4,Aakre Valga,Hummuli Valga,1,89,627213.008798677,6440209.31365088,619644.086140531,6418002.56618481,75386,470,2,POINT (627213.009 6440209.314)


In [401]:
# geodata_summer['KANT_start'].hist(figsize=(14,8),bins=100);

### Subseting data of main cities

In [402]:
# geodata_summer['KANT_end'].unique()

In [403]:
# subseting the main cities as DESTINATIONS Pärnu linn, Põhja-Tallinna linnaosa, Tartu linn, Narva linn
geoparnu = geodata_summer.loc[geodata_summer['KANT_end'] == 'Pärnu linn' ]
geotallinn = geodata_summer.loc[geodata_summer['KANT_end'] == 'Põhja-Tallinna linnaosa' ]
geotartu = geodata_summer.loc[geodata_summer['KANT_end'] == 'Tartu linn' ]
geonarva = geodata_summer.loc[geodata_summer['KANT_end'] == 'Narva linn' ]


In [404]:
cities = [geoparnu, geotallinn, geonarva, geotartu]

In [405]:
# iteration for coordinates

cities_wgs = []

for city in cities:
    
    # WGS origin
    city = city.to_crs(4326)
    
    # Origins Long and Lat
    city['lon_origin'] = [coor.x for coor in city['geometry'].to_list()]
    city['lat_origin'] = [coor.y for coor in city['geometry'].to_list()]
    
    # geometry destination CRS's
    xdestins = city['X_end'].to_list()
    ydestins = city['Y_end'].to_list()
    
    city = city.to_crs(3301)
    
    city['geometry'] = [Point(float(xcoor), float(ycoor)) for xcoor, ycoor in zip(xdestins, ydestins)]
    
    city = city.to_crs(4326)
    
    #Destinations Long and Lat
    city['lon_dest'] = [coor.x for coor in city['geometry'].to_list()]
    city['lat_dest'] = [coor.y for coor in city['geometry'].to_list()]
    
    city['route_id'] = city['route_id'].astype(float)
        
    cities_wgs.append(city)

In [406]:
geoparnu_summer = cities_wgs[0]
geotallinn_summer = cities_wgs[1]
geonarva_summer = cities_wgs[2]
geotartu_summer = cities_wgs[3]

# Now the same but in Winter

In [407]:
fps = r'data/OD_matrix_201601.csv'
geodata_winter = gpd.read_file(fps)

In [408]:
# define crs
geodata_winter.crs = CRS.from_epsg(3301)

In [409]:
geodata_winter.crs.name

'Estonian Coordinate System of 1997'

In [410]:
# adding geometry
xorigins = geodata_winter['X_start'].to_list()
yorigins = geodata_winter['Y_start'].to_list()

geodata_winter['geometry'] = [Point(float(xcoor), float(ycoor)) for xcoor, ycoor in zip(xorigins, yorigins)]

geodata_winter.head()

,KANT_start,KANT_end,start_kant_id,end_kant_id,X_start,Y_start,X_end,Y_end,route_id,Population,RegularMovers,geometry
0,Aakre Valga,Aakre Valga,1,1,627213.008798677,6440209.31365088,627213.008798677,6440209.31365088,1697,470,88,POINT (627213.009 6440209.314)
1,Aakre Valga,Elva linn,1,49,627213.008798677,6440209.31365088,642136.014360787,6456562.83692973,41506,470,10,POINT (627213.009 6440209.314)
2,Aakre Valga,Elva ümbruse kant Tartu,1,50,627213.008798677,6440209.31365088,639265.501559962,6453031.6043608,42353,470,1,POINT (627213.009 6440209.314)
3,Aakre Valga,Helme Valga,1,82,627213.008798677,6440209.31365088,611392.704776615,6429856.66892107,69457,470,2,POINT (627213.009 6440209.314)
4,Aakre Valga,Hummuli Valga,1,89,627213.008798677,6440209.31365088,619644.086140531,6418002.56618481,75386,470,1,POINT (627213.009 6440209.314)


In [411]:
# subseting the main cities as DESTINATIONS Pärnu linn, Põhja-Tallinna linnaosa, Tartu linn, Narva linn
geoparnu = geodata_winter.loc[geodata_summer['KANT_end'] == 'Pärnu linn' ]
geotallinn = geodata_winter.loc[geodata_summer['KANT_end'] == 'Põhja-Tallinna linnaosa' ]
geotartu = geodata_winter.loc[geodata_summer['KANT_end'] == 'Tartu linn' ]
geonarva = geodata_winter.loc[geodata_summer['KANT_end'] == 'Narva linn' ]

In [412]:
cities = [geoparnu, geotallinn, geonarva, geotartu]

In [413]:
# iteration for coordinates

cities_wgs = []

for city in cities:
    
    # WGS origin
    city = city.to_crs(4326)
    
    # Origins Long and Lat
    city['lon_origin'] = [coor.x for coor in city['geometry'].to_list()]
    city['lat_origin'] = [coor.y for coor in city['geometry'].to_list()]
    
    # geometry destination CRS's
    xdestins = city['X_end'].to_list()
    ydestins = city['Y_end'].to_list()
    
    city = city.to_crs(3301)
    
    city['geometry'] = [Point(float(xcoor), float(ycoor)) for xcoor, ycoor in zip(xdestins, ydestins)]
    
    city = city.to_crs(4326)
    
    #Destinations Long and Lat
    city['lon_dest'] = [coor.x for coor in city['geometry'].to_list()]
    city['lat_dest'] = [coor.y for coor in city['geometry'].to_list()]
    
    city['route_id'] = city['route_id'].astype(float)

        
    cities_wgs.append(city)

In [414]:
geoparnu_winter = cities_wgs[0]
geotallinn_winter = cities_wgs[1]
geonarva_winter = cities_wgs[2]
geotartu_winter = cities_wgs[3]

## Now the differences

In [415]:
# geoparnu_winter['route_str'] = geoparnu_winter['route_id'].astype(str)

In [416]:
# geoparnu_winter.dtypes

In [417]:
# match routes id
geoparnu_variation = geoparnu_summer.add_suffix('_s').merge(geoparnu_winter.add_suffix('_w'), left_on='route_id_s', right_on='route_id_w', how='outer')
geotallinn_variation = geotallinn_summer.add_suffix('_s').merge(geotallinn_winter.add_suffix('_w'), left_on='route_id_s', right_on='route_id_w', how='outer')
geonarva_variation = geonarva_summer.add_suffix('_s').merge(geonarva_winter.add_suffix('_w'), left_on='route_id_s', right_on='route_id_w', how='outer')
geotartu_variation = geotartu_summer.add_suffix('_s').merge(geotartu_winter.add_suffix('_w'), left_on='route_id_s', right_on='route_id_w', how='outer')


In [418]:
geoparnu_variation['RegularMovers_s'] = geoparnu_variation['RegularMovers_s'].fillna(0).astype(float)
geotallinn_variation['RegularMovers_s'] = geotallinn_variation['RegularMovers_s'].fillna(0).astype(float)
geonarva_variation['RegularMovers_s'] = geonarva_variation['RegularMovers_s'].fillna(0).astype(float)
geotartu_variation['RegularMovers_s'] = geotartu_variation['RegularMovers_s'].fillna(0).astype(float)

geoparnu_variation['RegularMovers_w'] = geoparnu_variation['RegularMovers_w'].fillna(0).astype(float)
geotallinn_variation['RegularMovers_w'] = geotallinn_variation['RegularMovers_w'].fillna(0).astype(float)
geonarva_variation['RegularMovers_w'] = geonarva_variation['RegularMovers_w'].fillna(0).astype(float)
geotartu_variation['RegularMovers_w'] = geotartu_variation['RegularMovers_w'].fillna(0).astype(float)

In [419]:
# geoparnu_variation.dtypes

In [420]:
geoparnu_variation['parnu_va'] = geoparnu_variation['RegularMovers_s']-geoparnu_variation['RegularMovers_w']
geotallinn_variation['tallinn_va'] = geotallinn_variation['RegularMovers_s']-geotallinn_variation['RegularMovers_w']
geonarva_variation['narva_va'] = geonarva_variation['RegularMovers_s']-geonarva_variation['RegularMovers_w']
geotartu_variation['tartu_va'] = geotartu_variation['RegularMovers_s']-geotartu_variation['RegularMovers_w']

### When it is closer to 0 no change. Negative, more in winter. Positive, more in summer

In [421]:
geoparnu_variation = geoparnu_variation[['lon_origin_s', 'lat_origin_s', 'lon_dest_s', 'lat_dest_s',
                                        'lon_origin_w', 'lat_origin_w', 'lon_dest_w', 'lat_dest_w',
                                        'parnu_va', 'RegularMovers_s', 'RegularMovers_w']]

geotallinn_variation = geotallinn_variation[['lon_origin_s', 'lat_origin_s', 'lon_dest_s', 'lat_dest_s',
                                        'lon_origin_w', 'lat_origin_w', 'lon_dest_w', 'lat_dest_w',
                                        'tallinn_va', 'RegularMovers_s', 'RegularMovers_w']]

geonarva_variation = geonarva_variation[['lon_origin_s', 'lat_origin_s', 'lon_dest_s', 'lat_dest_s',
                                        'lon_origin_w', 'lat_origin_w', 'lon_dest_w', 'lat_dest_w',
                                        'narva_va', 'RegularMovers_s', 'RegularMovers_w']]

geotartu_variation = geotartu_variation[['lon_origin_s', 'lat_origin_s', 'lon_dest_s', 'lat_dest_s',
                                        'lon_origin_w', 'lat_origin_w', 'lon_dest_w', 'lat_dest_w',
                                        'tartu_va', 'RegularMovers_s', 'RegularMovers_w']]

In [422]:
m = KeplerGl()

m.add_data(geoparnu_variation, 'Parnu visitors')
m.add_data(geotallinn_variation, 'Tallinn visitors')
m.add_data(geonarva_variation, 'Narva visitors')
m.add_data(geotartu_variation, 'Tartu visitors')

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [423]:
m.save_to_html(file_name='index.html')

Map saved to index.html!
